# cuML

我们要探索的另一个包是 [cuML](https://github.com/rapidsai/cuml)。与 `scikit-learn` 类似，你可以使用 `cuml` 在数据上训练机器学习模型来进行预测。与 RAPIDS 工具套件中的其他包一样，`cuml` 的 API 与 `scikit-learn` 相同，但底层代码已经实现为在 GPU 上运行。

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [1]:
import cudf

让我们来看看如何训练一个 K 近邻（K Nearest Neighbors）模型，根据血压、葡萄糖水平、BMI 等其他属性来预测某人是否患有糖尿病。

我们首先使用 `cudf` 将数据加载到 GPU 数据框中。

In [2]:
df = cudf.read_csv("/data/workdir/accelerated-computing-hub/gpu-python-tutorial/data/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


接下来，我们需要创建两个独立的表。一个包含除糖尿病列之外的患者属性，另一个只包含糖尿病列。

In [5]:
X = df.drop(columns=["Outcome"])
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [3]:
y = df["Outcome"].values
y[0:5]

array([1, 0, 1, 0, 1])

接下来，我们需要使用 `cuml` 中的 `train_test_split` 方法将数据分成两组。

第一个较大的集合将用于训练我们的模型。我们将从每个表中取 80% 的数据，并将它们称为 `X_train` 和 `y_train`。当模型被训练时，它将能够看到这两组数据以执行聚类。

另外 20% 的数据将被称为 `X_test` 和 `y_test`。一旦我们的模型训练完成，我们将把 `X_test` 数据输入到模型中，以预测这些人是否患有糖尿病。然后我们可以将这些预测与实际的 `y_test` 数据进行比较，以查看我们的模型有多准确。

我们还将 `random_state` 设置为 `1`，以使随机选择保持一致，这只是为了本教程的目的。我们还设置了 `stratify`，这意味着如果初始数据中 75% 的人患有糖尿病，那么训练集中也将保证有 75% 的人患有糖尿病。

In [6]:
from cuml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

现在我们有了训练数据，可以从 `cuml` 导入 `KNeighborsClassifier` 并拟合我们的模型。

In [7]:
from cuml.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)

knn.fit(X_train,y_train)

KNeighborsClassifier()

模型拟合在 GPU 上完成，现在我们可以进行一些预测。让我们预测测试集中的前五个人。

In [9]:
knn.predict(X_test)[0:5]

251    0
325    1
233    0
527    0
464    0
dtype: int64

我们可以看到，我们的新模型认为第一个患者患有糖尿病，但其余的人没有。

让我们将整个测试集连同实际答案一起通过评分函数运行，看看我们的模型表现如何。

In [10]:
knn.score(X_test, y_test)

0.6928104575163399

恭喜！你刚刚在 Python 中使用 GPU 训练了一个机器学习模型，并获得了 69% 的准确率。我们可以做很多事情来提高这个分数，但这超出了本教程的范围。